In [1]:
import math
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import f1_score

In [2]:
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src="/kaggle/input/mtc-best-weights/mi_model.py", dst="../working/mi_model.py")
copyfile(src="/kaggle/input/mtc-best-weights/ssvep_model.py", dst="../working/ssvep_model.py")

from mi_model import *
from ssvep_model import *

In [10]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

In [11]:
# Load models
mi_model    = EEGPipeline(eeg_channels=2, motion_channels=6, num_classes=2).to(device)
ssvep_model = EnhancedSSVEPModel(num_classes=4).to(device)

# Load weights
mi_checkpoint = torch.load("/kaggle/input/mtc-best-weights/mi_model_42.pth", map_location=device, weights_only=False)
mi_model.load_state_dict(mi_checkpoint['model_state_dict'])

ssvep_checkpoint = torch.load("/kaggle/input/mtc-best-weights/ssvep_model_epoch15_f10.6090_loss1.0770.pth", map_location=device, weights_only=False)
ssvep_model.load_state_dict(ssvep_checkpoint['model_state_dict'])

# Set models to evaluation mode
mi_model.eval()
ssvep_model.eval();

In [12]:
# Validation evaluation (same as before)
def evaluate_validation():
    val_mi_dataset = EEGDataset("/kaggle/input/mtcaic3/validation.csv", task='MI')
    val_ssvep_dataset = SSVEPDataset("/kaggle/input/mtcaic3/validation.csv", task='SSVEP')
    
    val_mi_loader = DataLoader(val_mi_dataset, batch_size=16, shuffle=False)
    val_ssvep_loader = DataLoader(val_ssvep_dataset, batch_size=16, shuffle=False)
    
    all_preds = []
    all_labels = []
    
    # Evaluate MI
    with torch.no_grad():
         for eeg_val, motion_val, labels_val, _ in val_mi_loader:
            eeg_val = eeg_val.permute(0, 2, 1).to(device)
            motion_val = motion_val.permute(0, 2, 1).to(device)
            labels_val = labels_val.to(device)
            
            outputs = mi_model(eeg_val, motion_val) 
            loss       = criterion(outputs, labels_val)
             
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels_val.cpu().numpy())
    
    mi_f1 = f1_score(all_labels, all_preds, average='macro')
    print(f"Validation MI F1 Score: {mi_f1:.4f}, Loss: {loss:.4f}")
    
    # Reset for SSVEP
    all_preds = []
    all_labels = []
    
    # Evaluate SSVEP
    with torch.no_grad():
        for eeg, freq, motion, labels, _ in val_ssvep_loader:
            eeg = eeg.to(device)
            freq = freq.to(device)
            motion = motion.to(device)
            labels = labels.to(device)
            
            outputs = ssvep_model(eeg, freq, motion)
            loss    = criterion(outputs, labels)
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    ssvep_f1 = f1_score(all_labels, all_preds, average='macro')
    print(f"Validation SSVEP F1 Score: {ssvep_f1:.4f}, Loss: {loss:.4f}")
    
    # Combined F1 score (weighted average)
    total_f1 = (mi_f1 * len(val_mi_dataset) + ssvep_f1 * len(val_ssvep_dataset)) / (len(val_mi_dataset) + len(val_ssvep_dataset))
    print(f"Combined Validation F1 Score: {total_f1:.4f}")

In [13]:
# Run validation evaluation
print("Running validation evaluation...")
evaluate_validation()

Running validation evaluation...
Validation MI F1 Score: 0.6667, Loss: 0.4698
Validation SSVEP F1 Score: 0.6090, Loss: 0.7597
Combined Validation F1 Score: 0.6378


In [14]:
# Corrected submission 
def create_submission():
    # Load test data separately for MI and SSVEP
    test_mi_dataset = EEGDataset("/kaggle/input/mtcaic3/test.csv", task='MI')
    test_ssvep_dataset = SSVEPDataset("/kaggle/input/mtcaic3/test.csv", task='SSVEP')
    
    test_mi_loader = DataLoader(test_mi_dataset, batch_size=16, shuffle=False)
    test_ssvep_loader = DataLoader(test_ssvep_dataset, batch_size=16, shuffle=False)
    
    mi_inverse_label_encoding = {
        0: 'Left',
        1: 'Right'
    }

    ssvep_inverse_label_encoding = {
        2: 'Left',
        3: 'Right',
        1: 'Forward',
        0: 'Backward'
    }
    
    # Dictionary to store predictions {id: label}
    predictions = {}
    
    # Process MI test data
    with torch.no_grad():
        for eeg, motion, _, ids in test_mi_loader:
            eeg    = eeg.permute(0, 2, 1).to(device)
            motion = motion.permute(0, 2, 1).to(device)
            outputs = mi_model(eeg, motion)
            preds = torch.argmax(outputs, dim=1)
            
            for id_num, pred in zip(ids, preds):
                predictions[id_num.item()] = mi_inverse_label_encoding[pred.item()]
    
    # Process SSVEP test data
    with torch.no_grad():
        for eeg, freq, motion, _, ids in test_ssvep_loader:
            eeg = eeg.to(device)
            freq = freq.to(device)
            motion = motion.to(device)
            
            logits = ssvep_model(eeg, freq, motion)
            preds = torch.argmax(logits, dim=1)
            
            for id_num, pred in zip(ids, preds):
                predictions[id_num.item()] = ssvep_inverse_label_encoding[pred.item()]
    
    # Create submission dataframe
    submission_df = pd.read_csv("/kaggle/input/mtcaic3/sample_submission.csv")
    
    # Fill in predictions in order of submission file
    for idx in submission_df['id']:
        submission_df.loc[submission_df['id'] == idx, 'label'] = predictions[idx]

    
    # Save submission
    submission_df.to_csv("submission.csv", index=False)
    print("\nSubmission file created: submission.csv")
    print(f"Total predictions made: {len(predictions)} (should be 100)")
    
    # Verify we have predictions for all test samples
    test_df = pd.read_csv("/kaggle/input/mtcaic3/test.csv")
    missing = set(test_df['id']) - set(predictions.keys())
    if missing:
        print(f"Warning: Missing predictions for IDs: {sorted(missing)}")
    else:
        print("Success: Predictions generated for all test samples")

In [15]:
# Create submission file
print("\nGenerating submission...")
create_submission()


Generating submission...

Submission file created: submission.csv
Total predictions made: 100 (should be 100)
Success: Predictions generated for all test samples
